You own a bike store. During week $t$, the (random) demand is $D_t$ units. 
On Monday morning you may choose to command $A_t$ additional units: they are delivered immediately before the shop opens. For each week:

  * Maintenance cost: $h$ per unit in stock left from the previous week (no maintenance is needed for newly commanded items)
  * Command cost: $c$ for each unit ordered + $c_0$ per command
  * Sales profit: $p$ per unit sold
  * Constraint: 
    - your warehouse has a maximal capacity of $M$ unit (any additionnal bike gets stolen)
    - you cannot sell bikes that you don't have in stock


* State: number of bikes in stock left from the last week => state space $\mathcal{S} = \{0,\dots, M\}$
* Action: number of bikes commanded at the beginning of the week => action space $\mathcal{A} = \{0, \dots ,M\}$ 
* Reward = balance of the week: if you command $a_t$ bikes,
$$r_t = -c_0 \mathbb{1}_{(a_t >0)}- c \times a_t - h\times s_t + p \times \min(D_t, s_t+a_t, M)$$
* Transitions: you end the week with the number of bikes $$s_{t+1} = \max\big(0, \min(M, s_t + a_t)  - D_t \big)$$ 

Our goal is to minimize the discounted sum of rewards, starting from an initial stock $s_1$, that is to find a policy whose value is 
$$V^*(s_1) = \max_{\pi}\mathbb{E}_{\pi}\left[\sum_{s=1}^{\infty} \gamma^{s-1}r_s\right].$$

In [ ]:
import numpy as np
import random as rd
import gym
from matplotlib import pyplot as plt
import time 

### Problem parameters 

In [ ]:
M = 15 # stock capacity
h = 0.3 # maintenance cost (per unit)
c = 0.5 # ordering cost (per unit)
c0 = 0.3 # fix delivery cost per command
p = 1 # selling price (per unit)

### Specifying the demand distribution 

The demand is modelled as a truncated geometric distribution. 

In [ ]:
# demand distribution (truncated geometric with parameter q)
q = 0.1
pdem = np.array([q*(1-q)**m for m in range(M+1)])
pdem[M] = pdem[M]+1-np.sum(pdem)

print("the average demand is ",np.sum([m*pdem[m] for m in range(M+1)]))

def SimuDemand(pdem): 
    cpdem = np.cumsum(pdem)
    i=0
    u = rd.random()
    while (u >cpdem[i]):
        i = i+1
    return i 

print("a simulated demand is ",SimuDemand(pdem))

### Encoding of the MDP as a gym environment

This is just to give an example on how gym works. 

In [ ]:
def nextstate(s,a,d,M):
    # computes the next state if the demand is d
    return max(0,min(M,s+a) -d)

def nextreward(s,a,d,M,c,c0,h,p):
    # computes the next state if the demand is d
    rew = -c*a - h*s + p*min(M,d,s+a)
    if (a>0):
        rew = rew - c0
    return rew

class StoreManagement(gym.Env):
    """
    Retail Store Management environment
    The environment defines which actions can be taken at which point and
    when the agent receives which reward.
    """
    def __init__(self,FirstState):
        
        # General variables defining the environment
        self.Stock_Capacity = M
        self.Maintenance_Cost = h
        self.Order_Cost = c 
        self.Delivery_Cost = c0
        self.Selling_Price = p
        self.Demand_Distribution = pdem
        
        # Define the action space
        self.action_space = gym.spaces.Discrete(self.Stock_Capacity+1)

        # Define the state space (state space = observation space in this example)
        self.observation_space = gym.spaces.Discrete(self.Stock_Capacity+1)

        # current time step
        self.curr_step = -1 # 
        
        # initial state
        self.state = FirstState

    def step(self, action):
        """
        simulates a transition following action a in the current state
        action : int
        """
        self.curr_step += 1
        # simulate the demand 
        Demand = SimuDemand(self.Demand_Distribution)
        # compute the reward
        reward = nextreward(self.state,action,Demand,self.Stock_Capacity,self.Order_Cost,self.Delivery_Cost,self.Maintenance_Cost,self.Selling_Price)
        # compute the next state 
        self.state = nextstate(self.state,action,Demand,self.Stock_Capacity)
        # return 4 elements: observation / reward / termination? (no terminal state here) / information (optional) 
        return self.state, reward, False, {}

    def reset(self,InitialStock):
        """
        Reset the state of the environment and returns an initial observation.
        Returns
        -------
        observation (object): the initial observation of the space.
        """
        self.curr_step = -1
        self.state = InitialStock
    
    def _render(self, mode='human'):
        """optional visualization of the interaction: none here"""
        return


### A function that simulates a trajectory under a policy Pi starting from some state $s_0$

In [ ]:
def SimulateTrajectory(T,Pi,s1):
    '''input: T length of the interaction, Pi a policy (function), s1 initial state'''
    Rewards = np.zeros(T+1)
    States = np.zeros(T+1)
    env=StoreManagement(s1)
    for t in range(T):
        States[t]=env.state
        action=Pi(env.state)
        state,rew,x,y=env.step(action)
        Rewards[t]=rew
    return States,Rewards

### Running simulations with three simple baselines 

In [ ]:
s1 = 10 # initial stock 
gamma = 0.97 # discount factor 

def PiUniform(s):
    # pick uniformly at random in {0,1,...,M-s}
    x = rd.sample(range(M+1-s),1)
    return s+x[0]

def PiConstant(s,c=3):
    # oder a constant number of c bikes 
    return min(c,M-s)

def PiThreshold(s,m1=4,m2=10):
    # if less than m1 bikes in stock, refill it up to m2
    action = 0
    if (s <=m1):
        action = (m2-s)
    return action
        

T = int(np.log(1/(0.001*(1-gamma)))/np.log(1/gamma)) # time needed for the value not to matter any more
States1,Rewards1 = SimulateTrajectory(T, PiUniform,s1)
States2,Rewards2 = SimulateTrajectory(T, PiConstant,s1)
States3,Rewards3 = SimulateTrajectory(T, PiThreshold,s1)

# without discount:
plt.plot(np.cumsum(Rewards1),label="random policy")
plt.plot(np.cumsum(Rewards2),label="constant policy")
plt.plot(np.cumsum(Rewards3),label="threshold policy")
plt.xlabel('weeks')
plt.ylabel('cumulated reward')
plt.legend()

# with discount:
plt.figure()
plt.plot(np.cumsum(Rewards1*np.array([gamma**t for t in range(T+1)])),label="random policy")
plt.plot(np.cumsum(Rewards2*np.array([gamma**t for t in range(T+1)])),label="constant policy")
plt.plot(np.cumsum(Rewards3*np.array([gamma**t for t in range(T+1)])),label="threshold policy")
plt.xlabel('weeks')
plt.ylabel('cumulated discounted reward')
plt.legend()

#plot(p1, p2, layout = (2,1), size = (800, 800))
#png("shop_run_profit.png")


### Evolution of the stock

In [ ]:
plt.plot(States1)
plt.xlabel('weeks')
plt.ylabel('stock')
plt.title('Evolution of the stock under a random policy')

plt.figure()
plt.plot(States2)
plt.xlabel('weeks')
plt.ylabel('stock')
plt.title('Evolution of the stock under a constant policy')


plt.figure()
plt.plot(States3)
plt.xlabel('weeks')
plt.ylabel('stock')
plt.title('Evolution of the stock under a threshold policy')

# Dynamic Programming

Here we assume that we work on a *known* MDP: that is, we know the demand distribution hence we are able to compute the parameters of the MDP: the transition kernel and the average reward

## Computation of the transitions and rewards 

In [ ]:
# Transition and Reward kernels
K = np.zeros((M+1,M+1,M+1)) # K[s,s',a] = p(s' | s,a) 
avgR = np.zeros((M+1,M+1)) # avgR[s,a] = average reward received in state s when playing action a

## computation: iterate over all possible states, actions, and possible demand values
for a in range(M+1):
    for s in range(M+1):
        for d in range(M+1):
            # next state and reward with demand d
            ns = max(0,min(M,s+a) -d)
            reward = -c*a - h*s+p*min(M,d,s+a)
            if (a>0):
                reward = reward - c0
            K[s,ns,a] += pdem[d]
            avgR[s,a] += pdem[d] * reward

## Evaluation of deterministic policies

First, by trying Monte-Carlo (that would also work for stochastic policies)

In [ ]:
## Computes the values in each state by Monte-Carlo simulation (needs many simulations to be precise)

Policy=PiThreshold

MC = 300 # number of Monte-Carlo simulations
Values = np.zeros(M+1)

for i in range (M+1):
    s1=i
    val = 0
    for j in range(MC):
        States,Rewards=SimulateTrajectory(T,Policy,s1)
        val += np.sum(Rewards*np.array([gamma**t for t in range(T+1)]))
    Values[i] = val/MC  

print("the estimated value in each states for the threshold policy are ")
print(Values)
    

Then by using the matrix inversion technique. **Complete the code below**

In [ ]:
def EvaluatePolicy(Pi):
    avgRPi=np.zeros((M+1,1))
    KPi=np.zeros((M+1,M+1))
    for s in range(M+1):
        KPi[s,:]=K[s,:,Pi(s)] # matrix P^\pi
        avgRPi[s]=avgR[s,Pi(s)] # vector r^\pi
    V = ## TO BE COMPLETED
    return V.reshape(M+1,)

Policy=PiThreshold
Values = EvaluatePolicy(Policy)

print(Values)
    

Comparing the threshold policy and the constant policy. 

In [ ]:
Values1 = EvaluatePolicy(PiConstant)
Values2 = EvaluatePolicy(PiThreshold)

plt.plot(Values1,label ="Constant policy c=3")
plt.plot(Values2,label = "Threshold policy m1=4, m2=10")
plt.xlabel('stock')
plt.ylabel('value ')
plt.legend()
plt.title('Evolution of the stock under a constant policy')


## Computing the Optimal Policy: Value and Policy Iteration

We can compare the optimal policy obtained with Value and Policy Iteration, and the runtime of the two algorithms. **For Policy Iteration, you need to complete the code below**. 

In [ ]:
# a function that performs policy improvement

def Improve(V):
    '''return Pi=greedy(V) and the value function of policy Pi'''
    Pi = np.zeros(M+1) # improved policy 
    newV = np.zeros(M+1)
    # compute the Q table 
    Q = np.zeros((M+1,M+1))
    for s in range(M+1):
        for a in range(M+1):
            Q[s,a]=avgR[s,a]+gamma*np.sum([K[s,ns,a]*V[ns] for ns in range(M+1)])
        # improvement (greedy policy wrt to Q)
        pi = np.argmax(Q[s,:])
        Pi[s]=pi
        newV[s]=Q[s,pi]
        Pi=Pi.astype(int)
    return Pi,newV


def ValueIteration(epsilon=1e-3): # epsilon = guaranteed precision
    # initialization 
    V = np.zeros(M+1)
    Pi = np.zeros(M+1)
    # new value function
    newV = 10*np.random.rand(M+1)
    nIt = 0
    while (np.linalg.norm(V-newV)>epsilon):
        nIt +=1
        V = np.copy(newV) 
        Pi,newV = Improve(V)
        #print("V is ",V," and newV is ",newV,"\n")
    return Pi,newV,nIt


def PolicyIteration():
    # initalization 
    Pi = np.zeros(M+1)
    V = np.zeros(M+1)
    # new policy 
    newPi = np.random.randint(M+1,size=M+1) 
    nIt = 0 
    while (np.linalg.norm(Pi-newPi)>0.1):
        nIt +=1 
        Pi = np.copy(newPi)
        # evaluate and improve policy Pi 
        newPi,newV = ### TO BE COMPLETED
    return Pi,V,nIt
        
start = time.time()
Pi,V,nIt = ValueIteration()
elapsed = time.time()-start

start = time.time()
Pi2,V2,nIt2 = PolicyIteration()
elapsed2 = time.time()-start

print("Value iteration yields policy",Pi,"and value ",V," in ",nIt," iterations and t=",elapsed," seconds\n")
print("Policy iteration yields ",Pi2,"and value ",V2," in ",nIt2," iterations and t=",elapsed2," seconds")


In [ ]:
plt.plot(Pi)
plt.xlabel('stock')
plt.ylabel('order under the optimal policy')
plt.title("Optimal Policy")

plt.figure()
plt.plot(V)
plt.xlabel('stock')
plt.ylabel('value of the optimal policy')
plt.title("Optimal Value")

Vstar=np.copy(V)

# Temporal Difference Methods

## Stochastic Approximation for Policy Evaluation: TD(0)

In [ ]:
def TD0(Pi, T):
    V = np.random.rand(M+1) # V[s] = estimated value of each state under policy pi
    N = np.zeros(M+1) # N[s] =number of visits to state s in the loop
    s0 = np.random.randint(M+1)
    env=StoreManagement(s0)
    for t in range(T):
        S = env.state
        N[S] += 1
        action = Pi(S)
        nS,rew,x,y = env.step(action) 
        alpha = 1/((1+N[S])**0.5) 
        V[S] = (1-alpha)*V[S] + alpha * (rew + gamma*V[nS]) 
    return(V)

T = 10**6
Pi = PiConstant

start=time.time()
print("value computed by TD0 ",TD0(Pi, T))
print("time elapsed is",time.time()-start,"\n")
print("value computed by matrix inversion",EvaluatePolicy(Pi))
# requires a large number of iterations to converge... 


## Q-learning

In [ ]:
def QLearning(Landmarks):
    star=time.time()
    nbLands = len(Landmarks)
    # random initialization of the Q table
    Q = 10*np.random.rand(1+M, 1+M) 
    N = np.zeros((M+1,M+1)) # number of visits to state-values 
    epsilon = 0.3
    s0 = np.random.randint(M+1)
    env=StoreManagement(s0)
    for k in range(nbLands-1):       
        for t in range(Landmarks[k]+1,Landmarks[k+1]):
            S=env.state
            A =np.random.randint(M+1)
            if (rd.random()>epsilon):# epsilon-greedy choice of action
                 A = np.argmax(Q[S, :]) # greedy action           
            N[S,A] += 1
            nS,rew,x,y = env.step(A)
            delta = rew + gamma * max(Q[nS, :]) - Q[S,A]
            alpha = 1/((1+N[S,A])**0.5)
            Q[S, A] += alpha * delta
        # compute the greedy policy 
        Pi = np.array([np.argmax(Q[s,:]) for s in range(M+1)])
        Pi = Pi.astype(int)
        print("After T=",Landmarks[k+1],"iterations (",np.floor(time.time()-start),"seconds), the policy is",Pi)
    return Pi,Q

Landmarks = np.array([0,10**3,10**4,10**5,10**6,10**7])
Pi,Q = QLearning(Landmarks)

# for T >= 10^7, we obtain correct policies
# for T >= 10^8, we almost obtain the right one